In [1]:
# PACKAGES
%reset -f
import numpy as np
import scipy as sc
from scipy import stats as st
import matplotlib.pyplot as plt
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C
import properscoring as ps
from IPython.display import clear_output
# from sklearn.calibration import calibration_curve
# from sklearn.svm import SVC
# from sklearn.naive_bayes import GaussianNB
# from sklearn.linear_model import LogisticRegression
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.isotonic import IsotonicRegression
# from statsmodels.distributions.empirical_distribution import ECDF
# import pandas as pd

In [2]:
# FUNCTIONS
def gp_predict(x_data,y_data):
    # Mesh the input space for multiple evaluations
    x_data = np.atleast_2d(x_data).T
    # Straighten out the observations array? Why did I have to do this?
    y_data = y_data.ravel()
    # Inisiate a Gaussian Process model
    # kernel = C(1.0, (1e-3, 1e3)) * RBF(10, (1e-2, 1e2)) # An obscenely overdetermined kernal if you want to hit every single point. But we don't.
    gp = GaussianProcessRegressor(n_restarts_optimizer=9)
    # Fit to data using Maximum Likelihood Estimation of the parameters
    gp.fit(x_data,y_data)
    # Make the prediction on the meshed x-axis (ask for MSE as well)
    y_pred, sigma = gp.predict(x_data, return_std=True)

    skew_predict_from_data = 0
    # skew_predict_from_data = 4
    y_pred+=skew_predict_from_data
    return y_pred, sigma

def plot_data_and_predictions(x_data,y_data,xf,yf,y_pred,title,legend_on):
    input_data = plt.scatter(x_data,y_data, color = 'k', facecolor = 'none' , label = 'Data')
    gp_prediction = plt.plot(x_data,y_pred,'b-' ,label = 'GP Prediction')
    true_function = plt.plot(xf,yf,'r-',label = 'True Function')
    if legend_on:
        plt.legend()
    plt.title(title, fontweight='bold')
def get_pdf(data):
    # Sample from a normal distribution using numpy's random number generator
    samples = data
    bin_min = np.min(data)
    bin_max = np.max(deta)
    n_bins = 20
    # Compute a histogram of the sample
    bins = np.linspace(bin_min,bin_max,n_bins)
    histogram, bins = np.histogram(samples, bins=bins, density=True)
    bin_centers = 0.5*(bins[1:] + bins[:-1])
    # Compute the PDF on the bin centers from scipy distribution object
    pdf = st.norm.pdf(bin_centers)
    return pdf

def reliability_curves(y_true,y_pred,num_bins):
    for ii in range(y_true.size):
        yt = y_true[ii]
        yp = y_pred[ii]
        
        prob_true, prob_pred = calibration_curve(yt, yp,normalize=true, n_bins=num_bins)
        plt.plot(prob_pred,prob_true,'o-')
    perfect_calibration = linspace(0,1,100)
    plt.plot(perfect_calibration,perfect_calibration,'k-')

def make_gaus_noisy_data(mu_f,sig_f):
    n_samples = mu_f.size
    y_data = np.random.normal(mu_f,sig_f,n_samples)
    return y_data


# RS IS BEING CALCULATED INCORRECTLY AS OF 6/17. THE FOLLOWING TWO FUNCTIONS NEED TO BE REWORKED TO CORRECT THIS.
# 
# standard_error()
# get_RS()
def standard_error(y_data,y_pred):
    epsilon = y_data - y_pred
    sigma = np.std(np.vstack((y_data,y_pred)),axis=0) 
    # THIS IS WRONG AS OF 6/17 !!!! CORRECT ASAP

    eta = epsilon / (np.sqrt(2)*sigma)
    return eta, sigma, epsilon
def get_RS(eta):
    phi = 0.5*(sc.special.erf(eta) + 1)
    C = np.empty((eta.size,))*0
    for ii in range(eta.size):
        C[ii] = np.mean(np.heaviside(eta[ii] - eta,eta*0))
    RS = np.sum(np.abs(phi - C)**2)
    N = eta.size
    ind = np.linspace(1,N)

    # Please note, at sufficiently high sample size (N) the RS_min will asymptote at RS_min = 0.4 

    RS_min = (1/(np.sqrt(np.pi)*N)) * np.sum( np.exp(-1*sc.special.erfinv( ((2*ind-1)/N) - 1)**2) - ((1/2)*np.sqrt(2/np.pi)) )
    return RS, RS_min
# 
# 
# 

def get_crps(y_data,y_pred):
    
    # Camporeale's analytic definition of CRPS (below) does not seem to EXACTLY recreate the CRPS in sklearn or properscoring but it is EXTREMELY close, on average within 3-5%.
    # I'm using his definition of CRPS in the paper for the sake of consistency.
    eta, sigma, epsilon = standard_error(y_data,y_pred)
    crps = sigma*( epsilon/sigma * sc.special.erf( epsilon/(np.sqrt(2)*sigma) ) + np.sqrt(2/np.pi)*np.exp( (-epsilon**2)/(2*sigma**2) ) - ( 1/np.sqrt(np.pi) ) )
    # properscoring's crps output is near identical to Camporeale's definition. So that's good.
    # crps = ps.crps_ensemble(y_data,y_pred)
    crps_min = (np.sqrt(np.log(4))/2)*np.mean(epsilon)
    return crps, crps_min

def get_BETA(RS_min,CRPS_min):
    Beta = RS_min/(CRPS_min+RS_min)
    return Beta

def get_ACCRUE(Beta,crps,RS):
    CRPS_mean = np.mean(crps)
    ACCRUE = Beta * CRPS_mean + (1-Beta) * RS
    return ACCRUE

def make_data_model_ensemble(xf,mu_f,sig_f,N_models):
    y_pred = np.empty((N_models,mu_f.size))
    y_true = np.empty((N_models,mu_f.size))
    pred_sigma = np.empty((N_models,mu_f.size))
    for ii in range(N_models):
        yt = make_gaus_noisy_data(mu_f,sig_f)
        yp,sigma = gp_predict(xf,yt)
        y_pred[ii] = yp
        y_true[ii] = yt
        pred_sigma[ii] = sigma
    y_pred = np.squeeze(y_pred)
    y_true = np.squeeze(y_true)
    return y_pred, pred_sigma, y_true

def force_prediction_errors(y_pred,errors):
    N = y_predict_ensemble.shape
    N = N[0]
    for ii in range(N):
        y_pred[ii] = y_pred[ii] + errors[ii]
    return y_pred
    
def camporeale_toy_problems(name,N_points):
    if name == 'W':
        # W toy problem
        x_min = 0
        x_max = np.pi
        xf = np.linspace(x_min,x_max,int(N_points))
        yf = np.sin(2.5*xf)*np.sin(1.5*xf)
        mu_f = yf
        sig_f = 0.01 + 0.25*(1-np.sin(2.5*xf))**2
        title = 'W'
        y_data = make_gaus_noisy_data(mu_f,sig_f)
        y_pred,sigma = gp_predict(xf,y_data)
    elif name == 'Y':
        # Y toy problem
        x_min = 0
        x_max = 1; 
        xf = np.linspace(x_min,x_max,int(N_points))
        yf = 2*(np.exp(-30*(xf-0.25)**2) + np.sin(np.pi*xf**2)) - 2
        mu_f = yf
        sig_f = np.exp(np.sin(2*np.pi*xf))/3
        title = 'Y'
        y_data = make_gaus_noisy_data(mu_f,sig_f)
        y_pred,sigma = gp_predict(xf,y_data)
    elif name == 'G':
        # G toy problem
        x_min = 0
        x_max = 1
        xf = np.linspace(x_min,x_max,int(N_points))
        yf = 2*np.sin(2*np.pi*xf)
        mu_f = yf
        sig_f = 0.5*xf+0.5
        title = 'G'
        y_data = make_gaus_noisy_data(mu_f,sig_f)
        y_pred,sigma = gp_predict(xf,y_data)
    elif name == '5D':
        # 5D toy problem
        x_min = 0
        x_max = 1
        N_points = 7 # N_points is enforced at 7 points as a more complex vector will stress a standard machine when meshed over 5 dimensions, just for this specific toy problem.
        # Make a 5-dimensional matrix where the columns comprise every possible permutation of the variables within the 5-D space
        xf = np.linspace(x_min,x_max,N_points); x1,x2,x3,x4,x5 = np.meshgrid(xf,xf,xf,xf,xf);xf_meshed = np.vstack((x1.ravel(),x2.ravel(),x3.ravel(),x4.ravel(),x5.ravel()));del x1,x2,x3,x4,x5
        xf = xf_meshed[4,:] # got to choose a 1-D axis for this toy problem as Camporeale reduces this 5-D problem to 1-D analysis.
        sig_f = 0.45*(np.cos(np.pi + np.sum(xf_meshed*5,axis=0)) + 1.2)
        yf = sig_f*0
        mu_f = yf
        title = '5D Toy Problem'
        y_data = 0 #make_gaus_noisy_data(mu_f,sig_f)
        y_pred = 0
        sigma =  0 #gp_predict(xf,y_data)

    title = title
    x_data = xf
    y_true = yf
    y_data = y_data
    y_pred = y_pred
    sig_true = sig_f
    sig_pred = sigma
    return x_data, y_data, y_true, y_pred, sig_true, sig_pred, title


# end functions

In [3]:
# del y_data, y_pred, eta, sigma, epsilon, CRPS, CRPS_min, RS, RS_min, Beta, ACCRUE, RS_vec, CRPS_vec
# Toy_Problems = 'W'
# Toy_Problems = 'G'
# Toy_Problems = 'Y'
Toy_Problems = ['W','G','Y']
N_models = 500
N_points = 100 # per model
N_problems = len(Toy_Problems)

In [4]:
fig,axs = plt.subplots(N_problems,2)
fig.set_size_inches([15,15])
row=-1
for jj in range(N_problems):
    row+=1
    curr_Toy_Problem = Toy_Problems[jj]
    x_data, y_data, y_true, y_pred, sig_true, sig_pred, title = camporeale_toy_problems(curr_Toy_Problem,N_points)
    y_predict_ensemble, ensemble_sigma, y_data_ensemble = make_data_model_ensemble(x_data,y_true,sig_true,N_models)
    clear_output(wait=True)
    ACCRUE = np.empty((N_models,))*0; RS_vec = np.empty((N_models,))*0; CRPS_vec = np.empty((N_models,))*0


    y_predict_ensemble = force_prediction_errors(y_predict_ensemble,ensemble_sigma)


    for ii in range(N_models):
        y_data = y_data_ensemble[ii]
        y_pred = y_predict_ensemble[ii]
        eta, sigma, epsilon = standard_error(y_data,y_pred)
        crps, CRPS_min = get_crps(y_data,y_pred)
        RS, RS_min = get_RS(eta)
        Beta = get_BETA(RS_min,CRPS_min)
        ACCRUE[ii] = get_ACCRUE(Beta,crps,RS)
        RS_vec[ii] = RS
        CRPS_vec[ii] = np.mean(crps)
    clear_output(wait=True)


    # sort_ind = np.argsort(ACCRUE)
    # ACCRUE = ACCRUE[sort_ind]
    # CRPS_vec = CRPS_vec[sort_ind]
    # RS_vec = RS_vec[sort_ind]

    if N_problems>1:
        plt.axes(axs[row,0])
    else:
        plt.axes(axs[0])

    for ii in range(N_models):
        plot_data_and_predictions(x_data,y_data_ensemble[ii],x_data,y_true,y_predict_ensemble[ii],'model/data ensemble: ' + title + '  ' + str(N_models) + ' model(s)',0)

    if N_problems>1:
        plt.axes(axs[row,1])
    else:
        plt.axes(axs[1])

    scatterpl = plt.scatter(CRPS_vec,RS_vec,c = ACCRUE)
    scatterpl.set_clim(np.min(ACCRUE),np.max(ACCRUE))
    plt.xlabel('Continuous Ranked Probability Score (CRPS)', fontweight='bold')
    plt.ylabel('Reliability Score (RS)', fontweight='bold')
    plt.title(title + ' toy problem - Model Ensemble - ' + str(N_models) + ' model(s)', fontweight='bold')
    cbr = plt.colorbar(scatterpl)
    cbr.set_label(label='ACCRUE Score',fontweight='bold')


    # Some notes:
    #
    # In the case of these toy problems, which are all gaussian distributed random data (aka symmetricly distributed statistically), the use of CRPS in the ACCRUE scalar seems to effectively discriminate better definitions of CRPS from other models in the ensembles shown.

    # It appears the same cannot be said of the RS score, however. Atleast with the caxis we use here, there shows no capacity to discriminate better RS scores using ACCRUE. In example, the ACCRUE values for a given CRPS (x-axis) seem to be far too similar across all RS scores values (y-axis) at that given CRPS.
    # CAUSES:
    # (1) Again, the MOST LIKELY cause is I'm just doing something wrong (probably with RS). I'm pretty new to this.
    # 
    # (2) The inability to discriminate RS using ACCRUE may just be a consequence of the symmetry of the observations (black dots) about the true function (red line). 
    #       -I could test this by applying a static shift to the predictions.
    #       ---If the RS score then begins to show a bias after doing so (using ACCRUE) this will increase the spread of the predictions around the true function shich could indicate RS as it is defined ay NOT work for gaussian distributed data or at the very least it needs to be tunes (with Beta) to be more sensitive.
    #       ----If that continues to be a developing argument then a better definition for RS is likely required.
    #       ----To that end, if an ideal realibility diagram is a perfectly straight line (CDF==PDF) then a scalar quantity that would more intuitively and more exactly reflect the divergence of the realibility from that straight line may be the way to go, such as R^2 or chi-squared.
    #       ---If the RS score behavior persists with no change then
    #           ---....?
    # 
    # 
    #
    # PRIORITY-->          ---After testing, it seems to be the FIRST scenario. I fucked something up. Only by forcing more variability into the ensemble of predictions can I get the RS score to dramatically discriminate better RS and CRPS values using ACCRUE. When I don't do this, it is too subtle to see in this plot which is not the same as saying it is too subtle to use for discrimination numerically. I need to investigate this more by binning RS scores within windows for small ranges of CRPS.
